In [2]:
from bm_preproc import BoyerMoore as BM
from kmer_index import Index as idx

In [42]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2022-08-23 21:12:24--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.32.118.211, 13.32.118.217, 13.32.118.19, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.32.118.211|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta.1’

chr1.GRCh38.excerpt 100%[===================>] 791,12K  --.-KB/s    in 0,1s    

2022-08-23 21:12:25 (5,16 MB/s) - ‘chr1.GRCh38.excerpt.fasta.1’ saved [810105/810105]



In [3]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [4]:
genome = readGenome("chr1.GRCh38.excerpt.fasta")
genome

'...'

In [5]:
def naive(p, t):
    occurrences = []
    count_chars = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            count_chars += 1
            #print(chars)
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, count_chars

In [6]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    count_align = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        count_align += 1
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, count_align

In [32]:
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"

In [33]:
# how many alignments does naive try? 

align_nu = len(genome)-len(p)+1
align_nu

799954

In [34]:
# how many a does naive try? 

_, chars = naive (p, genome)
chars

984143

In [35]:
p_bm = BM(p)
_, count_align = boyer_moore (p, p_bm, genome)
count_align

127974

In [11]:
def approximate_match(p, t, n):
    #m-start+len(p) or m-start+j: with m-start you go back i*segment_length times which corresponds to the offset
    #T possibly matches with P[0]
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    hits = 0
    p_idx = idx(t, segment_length)#define class or instantiate
    for i in range(n+1): 
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = p_idx.query(p[start:end]) #matches in only one segment, then instantiate another variable through method
        # Extend matching segments to see if whole p matches
        for m in matches:
            hits +=1 #all hits
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), hits

In [36]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
n=2
all_matches, hits = approximate_match(p, genome, n)
print(len(all_matches), hits)

19 90


In [13]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [22]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [23]:
p = 'TTATAT'
print(ind.query(p[0:]))

[]


In [24]:
p = 'TTATAT'
print(ind.query(p[1:]))

[1]


In [25]:
def approximate_match_subseq(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    ival = n+1
    all_matches = set()
    hits = 0
    p_subseq_idx = SubseqIndex(t, segment_length, ival) #define class or instantiate
    for i in range(n+1): 
        start = i
        matches = p_subseq_idx.query(p[start:]) #loops over all subsequences
        # Extend matching segments to see if whole p matches
        for m in matches:
            hits +=1 #all hits
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), hits

In [26]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
n=2
_,hits = approximate_match_subseq(p,genome,n)
hits

79